# data_d25a.ipynb
1. Time series: for global mean, relative, and geocentric at gauges, save fusion, low, central, high, and high-end projections for 2020–2100.
2. Year-2100: low, central, high, and high-end projections for gauge/grid locations and cities.

Author: Benjamin S. Grandey.

In [1]:
import d25a
import datetime

In [2]:
# Get start datetime
start_dt = datetime.datetime.now()

In [3]:
# Print package versions
print(d25a.get_watermark())

Python implementation: CPython
Python version       : 3.10.16
IPython version      : 8.31.0

matplotlib: 3.10.0
numpy     : 2.2.2
pandas    : 2.2.3
seaborn   : 0.13.2
xarray    : 2025.1.1

conda environment: d25a-rsl-fusion

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 22.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



## 1. Time series of global mean, relative, and geocentric sea level for fusion, low, central, high, and high-end projections

In [4]:
# Loop over global mean, relative, and geocentric sea level
for slr_str in ('gmsl', 'rsl', 'novlm'):
    # Loop over fusion, low etc projections
    for proj_str in ('fusion-ssp585', 'fusion-ssp126', 'low-end', 'low', 'central', 'high', 'high-end'):
        # Get and write projection time series
        _ = d25a.write_time_series_da(slr_str=slr_str, proj_str=proj_str)

Writing time_series/gmsl_fusion-ssp585_d25a.nc
Writing time_series/gmsl_fusion-ssp126_d25a.nc
Writing time_series/gmsl_low-end_d25a.nc
Writing time_series/gmsl_low_d25a.nc
Writing time_series/gmsl_central_d25a.nc
Writing time_series/gmsl_high_d25a.nc
Writing time_series/gmsl_high-end_d25a.nc
Writing time_series/rsl_fusion-ssp585_d25a.nc (1631 locations)
Writing time_series/rsl_fusion-ssp126_d25a.nc (1631 locations)
Writing time_series/rsl_low-end_d25a.nc (1631 locations)
Writing time_series/rsl_low_d25a.nc (1631 locations)
Writing time_series/rsl_central_d25a.nc (1631 locations)
Writing time_series/rsl_high_d25a.nc (1631 locations)
Writing time_series/rsl_high-end_d25a.nc (1631 locations)
Writing time_series/novlm_fusion-ssp585_d25a.nc (1631 locations)
Writing time_series/novlm_fusion-ssp126_d25a.nc (1631 locations)
Writing time_series/novlm_low-end_d25a.nc (1631 locations)
Writing time_series/novlm_low_d25a.nc (1631 locations)
Writing time_series/novlm_central_d25a.nc (1631 locations)

In [5]:
# Get and write locations information DataFrame to CSV
_ = d25a.write_locations_info_df()
# Read locations
locations_info_df = d25a.read_locations_info_df()
locations_info_df.head()

Writing time_series/locations_info_d25a.csv (1631 locations)
Reading time_series/locations_info_d25a.csv


,lat,lon,gauge_id,gauge_name,gauge_country
location,,,,,
1,48.38,-4.49,1,BREST,FRANCE
2,53.92,14.23,2,SWINOUJSCIE,POLAND
3,51.45,0.74,3,SHEERNESS,UNITED KINGDOM
5,53.31,-4.62,5,HOLYHEAD,UNITED KINGDOM
7,53.87,8.72,7,CUXHAVEN_2,GERMANY


## 2. Year-2100 projections for gauge/grid locations and cities

In [6]:
for slr_str in ('rsl', 'novlm'):
    for gauges_str in ('gauges', 'grid'):
        for cities_str in (None, 'cities', 'megacities'):
            _ = d25a.write_year_2100_df(slr_str=slr_str, gauges_str=gauges_str, cities_str=cities_str)

Reading time_series/locations_info_d25a.csv
Reading time_series/rsl_low-end_d25a.nc
Reading time_series/rsl_low_d25a.nc
Reading time_series/rsl_central_d25a.nc
Reading time_series/rsl_high_d25a.nc
Reading time_series/rsl_high-end_d25a.nc
Writing year_2100/rsl_gauges_2100_d25a.csv (1016 locations)
Reading time_series/locations_info_d25a.csv
Reading time_series/rsl_low-end_d25a.nc
Reading time_series/rsl_low_d25a.nc
Reading time_series/rsl_central_d25a.nc
Reading time_series/rsl_high_d25a.nc
Reading time_series/rsl_high-end_d25a.nc
430 out of 876 cities have a gauges rsl projection
Writing year_2100/rsl_gauges_cities_2100_d25a.csv (430 locations)
Reading time_series/locations_info_d25a.csv
Reading time_series/rsl_low-end_d25a.nc
Reading time_series/rsl_low_d25a.nc
Reading time_series/rsl_central_d25a.nc
Reading time_series/rsl_high_d25a.nc
Reading time_series/rsl_high-end_d25a.nc
33 out of 48 megacities have a gauges rsl projection
Writing year_2100/rsl_gauges_megacities_2100_d25a.csv (3

In [7]:
# Get end datetime
end_dt = datetime.datetime.now()
# Calculate run timedelta
run_td = end_dt - start_dt
# Print timing information
print(f"Start:     {start_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"End:       {end_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Wall time: {run_td.seconds} s")

Start:     2025-07-24 17:36:41
End:       2025-07-24 17:38:26
Wall time: 105 s
